In [3]:
import sys
import pandas as pd
sys.path.append('..')

from ai4mat.data.data import (
    read_structures_descriptions,
    read_defects_descriptions,
    StorageResolver,
    TEST_FOLD,
    TRAIN_FOLD,
)

In [4]:
train_dataset = "high_density_defects/MoS2_500"
test_dataset = "low_density_defects/MoS2"

In [6]:
storage_resolver = StorageResolver()
train_structures = read_structures_descriptions(storage_resolver['csv_cif']/train_dataset)
test_structures = read_structures_descriptions(storage_resolver['csv_cif']/test_dataset)
test_defects = read_defects_descriptions(storage_resolver['csv_cif']/test_dataset)

In [10]:
test_defect_id = test_defects[test_defects.defects.apply(lambda d: d == [{'type': 'vacancy', 'element': 'Mo'}, {'type': 'vacancy', 'element': 'S'}])].index[0]
print(test_defect_id)

6141cf1102d926221cabc543


In [12]:
test_structures = test_structures[test_structures.descriptor_id == test_defect_id]

In [16]:
folds = pd.Series(index=train_structures.index.union(test_structures.index), name='fold', data=TRAIN_FOLD)
folds.loc[test_structures.index] = TEST_FOLD

In [20]:
experiment_path = storage_resolver['experiments'].joinpath('MoS2_high_to_V2')

In [21]:
experiment_path.mkdir(exist_ok=True)

In [22]:
folds.to_csv(experiment_path / 'folds.csv.gz', index_label='_id')

In [23]:
import yaml
config = {"datasets": [train_dataset, test_dataset],
          "strategy": "train_test",
          "n-folds": 2,
          "targets": ["formation_energy_per_site"]}
with open(experiment_path.joinpath("config.yaml"), "wt") as config_file:
    yaml.dump(config, config_file)